In [ ]:
#Note: this code is also found in trio_pilot_pca; it is documented here to describe how the mt ref file was created

In [ ]:
import hail as hl
import gwaspy.pca as pca

hl.init()

In [ ]:
#create AGVP reference data
agvp = hl.import_plink(bed='gs://nd_popgen/agvp/AGVP.postQC.autosomes.bed',
                     bim='gs://nd_popgen/agvp/AGVP.postQC.autosomes.bim',
                    fam='gs://nd_popgen/agvp/AGVP.postQC.autosomes.fam',
                     reference_genome='GRCh37')

In [ ]:
#liftover to grch38: create coordinates from 37:38
rg37 = hl.get_reference('GRCh37')  
rg38 = hl.get_reference('GRCh38')  
rg37.add_liftover('gs://hail-common/references/grch37_to_grch38.over.chain.gz', rg38)  

In [ ]:
agvp.show()

In [ ]:
agvp = agvp.annotate_rows(new_locus=hl.liftover(agvp.locus, 'GRCh38', include_strand=True),
                 old_locus=agvp.locus)  
agvp = agvp.filter_rows(hl.is_defined(agvp.new_locus) & ~agvp.new_locus.is_negative_strand)  
agvp = agvp.key_rows_by(locus=agvp.new_locus.result, alleles = agvp.alleles)  

In [ ]:
#save the agvp file
agvp.write('gs://nd_popgen/agvp/agvp_grch38_final.mt')

In [ ]:
#also save plink versions of agvp grch38 file
agvp = hl.read_matrix_table('gs://nd_popgen/agvp/agvp_grch38_final.mt')
hl.export_plink(agvp, 'gs://nd_popgen/agvp/agvp_grch38_plink', ind_id = agvp.s)